In [1]:
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
import myfm
from myfm import RelationBlock
import pandas as pd
from scipy import sparse as sps
# read movielens 100k data.
from myfm.utils.benchmark_data import MovieLens100kDataManager

In [2]:
data_manager = MovieLens100kDataManager()
df_train, df_test = data_manager.load_rating(fold=1) # fold 1 is the toughest one

user_info = data_manager.load_userinfo().set_index('user_id')
user_info['age'] = user_info.age // 5 * 5
user_info['zipcode'] = user_info.zipcode.str[0]
user_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(user_info)

movie_info, movie_genres = data_manager.load_movieinfo()
movie_info['release_year'] = [
    str(x) for x in movie_info['release_date'].dt.year.fillna('NaN')
] # hack to avoid NaN
movie_info = movie_info[['movie_id', 'release_year'] + movie_genres].set_index('movie_id')
movie_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(movie_info.drop(columns=movie_genres))

date_be = OneHotEncoder(handle_unknown='ignore').fit(
    df_train.timestamp.dt.date.values.reshape(-1, 1)
)
def categorize_date(df):
    return date_be.transform(df.timestamp.dt.date.values[:, np.newaxis])

In [3]:
user_info.head()

,age,gender,occupation,zipcode
user_id,,,,
1,20,M,technician,8
2,50,F,other,9
3,20,M,writer,3
4,20,M,technician,4
5,30,F,other,1


In [4]:
movie_info.head()

,release_year,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,
1,1995.0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1995.0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1995.0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,1995.0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [5]:
# index "0" is reserved for unknown ids.
user_to_index = defaultdict(lambda : 0, { uid: i+1 for i,uid in enumerate(np.unique(df_train.user_id)) })
movie_to_index = defaultdict(lambda: 0, { mid: i+1 for i,mid in enumerate(np.unique(df_train.movie_id))})
USER_ID_SIZE = len(user_to_index) + 1
MOVIE_ID_SIZE = len(movie_to_index) + 1

In [6]:
# Implement side information and flavor of SVD++
# We add "all users who have evaluated a movie in the train set" or
# "all movies rated by a user" as a feture of user/movie.
use_date = True # use date info or not
use_iu = True # use implicit user feature
use_ii = True # use implicit item feature
use_user_info = True # use user information
use_movie_info = True # use movie information

movie_vs_watched = dict()
user_vs_watched = dict()
for row in df_train.itertuples():
    user_id = row.user_id
    movie_id = row.movie_id
    movie_vs_watched.setdefault(movie_id, list()).append(user_id)
    user_vs_watched.setdefault(user_id, list()).append(movie_id)

if use_date:
    X_date_train = categorize_date(df_train)
    X_date_test  = categorize_date(df_test)
else:
    X_date_train, X_date_test = (None, None)

In [7]:
# setup grouping
feature_group_sizes = []
if use_date:
    feature_group_sizes.append(
        len(date_be.categories_[0]), # date
    )

feature_group_sizes.append(USER_ID_SIZE) # user ids

if use_iu:
    feature_group_sizes.append(MOVIE_ID_SIZE)

if use_user_info:
    feature_group_sizes.extend([
        len(c) for c in user_info_ohe.categories_ # user attributes
    ])

feature_group_sizes.append(MOVIE_ID_SIZE) # movie ids
                           
if use_ii:
    feature_group_sizes.append(USER_ID_SIZE)

if use_movie_info:
    feature_group_sizes.extend([
        len(c) for c in movie_info_ohe.categories_ # user attributes
    ])
    feature_group_sizes.append(len(movie_genres))


In [8]:
# given user/movie ids, add additional infos and return it as sparse
def augment_user_id(user_ids):
    Xs = []
    X_uid = sps.lil_matrix((len(user_ids), USER_ID_SIZE))
    for index, user_id in enumerate(user_ids):
        X_uid[index, user_to_index[user_id]] = 1
    Xs.append(X_uid)
    if use_iu:
        X_iu = sps.lil_matrix((len(user_ids), MOVIE_ID_SIZE))
        for index, user_id in enumerate(user_ids):
            watched_movies = user_vs_watched.get(user_id, [])
            normalizer = 1 / max(len(watched_movies), 1) ** 0.5
            for uid in watched_movies:
                X_iu[index, movie_to_index[uid]] = normalizer
        Xs.append(X_iu)
    if use_user_info:
        Xs.append(user_info_ohe.transform(user_info.reindex(user_ids)))
    return sps.hstack(Xs, format='csr')

def augment_movie_id(movie_ids):
    Xs = []
    X_movie = sps.lil_matrix((len(movie_ids), MOVIE_ID_SIZE))
    for index, movie_id in enumerate(movie_ids):
        X_movie[index, movie_to_index[movie_id]] = 1
    Xs.append(X_movie)
    
    if use_ii:
        X_ii = sps.lil_matrix((len(movie_ids), USER_ID_SIZE))
        for index, movie_id in enumerate(movie_ids):
            watched_users = movie_vs_watched.get(movie_id, [])
            normalizer = 1 / max(len(watched_users), 1) ** 0.5
            for uid in watched_users:
                X_ii[index, user_to_index[uid]] = normalizer
        Xs.append(X_ii)    
    
    if use_movie_info:
        Xs.append(movie_info_ohe.transform(movie_info.drop(columns=movie_genres).reindex(movie_ids)))
        Xs.append(sps.csr_matrix(movie_info.reindex(movie_ids)[movie_genres].values))
    return sps.hstack(Xs, format='csr')

## User Relation Block to express data
See [\[Rendle 2013\]](http://www.vldb.org/pvldb/vol6/p337-rendle.pdf) how comlexity dcrease drastically in this case (and most cases with bipartite graph structure).

In [9]:
# Create RelationBlock.
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html
train_blocks = []
test_blocks = []
for source, target in [(df_train, train_blocks), (df_test, test_blocks)]:
    unique_users, user_map = np.unique(source.user_id, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movie_id, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

## Regression

In [10]:
fm = myfm.MyFMRegressor(rank=10)
fm.fit(
    X_date_train, df_train.rating.values, X_rel=train_blocks,
    group_shapes=feature_group_sizes, X_test=X_date_test, X_rel_test=test_blocks,
    y_test=df_test.rating.values,
    n_iter=300, n_kept_samples=295
);

alpha = 1.62 w0 = 3.70  rmse_this: 0.96 mae_this: 0.76: 100%|██████████| 300/300 [00:14<00:00, 21.08it/s]


In [11]:
test_predictions = fm.predict(X_date_test, test_blocks)

rmse = (
    (test_predictions - df_test.rating.values)**2
).mean() ** 0.5
mae = np.abs(test_predictions - df_test.rating).mean()

# Note the improvement from "id_only" case.
# Compare this with methods like ones in https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k
print('rmse={}, mae={}'.format(rmse, mae))

rmse=0.8843810104515312, mae=0.6950517128324772


## Ordered Probit Regression

In [12]:
fm_probit = myfm.MyFMOrderedProbit(rank=10)
fm_probit.fit(
    X_date_train, df_train.rating.values - 1, X_rel=train_blocks,
    group_shapes=feature_group_sizes,
    n_iter=300, n_kept_samples=295
);

w0= -0.013666, cutpoint = ['-2.461', '-1.502', '-0.269', '1.167'] : 100%|██████████| 300/300 [00:29<00:00, 10.17it/s]


In [13]:
test_prediction_ordered_prob = fm_probit.predict_proba(X_date_test, test_blocks)
test_prediction_ordered_mean = 1 + test_prediction_ordered_prob.dot(np.arange(5)) # class 0 => rating 1 shift

rmse = (
    (test_prediction_ordered_mean - df_test.rating.values) **2
).mean() ** 0.5
mae = np.abs(test_prediction_ordered_mean - df_test.rating).mean()
test_predictions = fm.predict(X_date_test, test_blocks)

print('rmse={}, mae={}'.format(rmse, mae))

rmse=0.8819410811182534, mae=0.6928870158692445


In [14]:
# If we use the original data format, it takes much more!
X_original_format = []
if use_date:
    X_original_format.append(X_date_train)

X_original_format.extend(
    [rel.data[rel.original_to_block] for rel in train_blocks]
)

X_original_format = sps.hstack(X_original_format, format='csr')

fm_rawformat = myfm.MyFMRegressor(rank=10).fit(X_original_format, df_train.rating,
                                          group_shapes=feature_group_sizes, n_iter=6, n_kept_samples=1)

alpha = 1.31 w0 = 3.45 : 100%|██████████| 6/6 [00:12<00:00,  2.02s/it]


In [15]:
# They shoud be same up to floating point artifact.
fm_rawformat.predictor_.samples[-1].V - fm.predictor_.samples[0].V

array([[ 2.33146835e-15,  5.96744876e-16, -4.44089210e-16, ...,
         2.02615702e-15, -1.08246745e-15,  3.60822483e-16],
       [-3.09474668e-15, -2.44249065e-15,  5.24580379e-15, ...,
         6.96491476e-16, -2.08860707e-15,  2.28983499e-15],
       [-1.19348975e-15, -1.60982339e-15,  1.59594560e-15, ...,
        -1.77982629e-15,  1.07552856e-15, -1.63757896e-15],
       ...,
       [-5.82867088e-16, -5.55111512e-17,  1.09634524e-15, ...,
        -1.51267887e-15, -1.22124533e-15,  1.04083409e-16],
       [ 3.88578059e-16,  1.38777878e-17,  1.23512311e-15, ...,
        -1.47104551e-15, -2.25080371e-16,  4.30211422e-16],
       [ 9.99200722e-16,  1.52655666e-15, -1.14144805e-15, ...,
         1.91513472e-15, -1.55431223e-15, -2.87270208e-15]])